<a href="https://colab.research.google.com/github/Harmonai-org/sample-generator/blob/main/Finetune_Dance_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dance Diffusion finetune

Licensed under the MIT License

Copyright (c) 2022 Zach Evans

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.


# Set Up

In [1]:
#@title Check GPU Status
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
    #!nvidia-smi
    nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
else:
    #!nvidia-smi -i 0 -e 0
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_ecc_note)

GPU 0: NVIDIA GeForce RTX 3080 (UUID: GPU-f2e7b236-c1d4-1140-c118-bbc088036f4a)



In [2]:
#@title Prepare folders
import subprocess, os, sys, ipykernel

def gitclone(url, targetdir=None):
    if targetdir:
        res = subprocess.run(['git', 'clone', url, targetdir], stdout=subprocess.PIPE).stdout.decode('utf-8')
    else:
        res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipi(modulestr):
    res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipie(modulestr):
    res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def wget(url, outputdir):
    res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

#todo remove colab

try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/Bass_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_audio'
createPath(initDirPath)
outDirPath = f'{root_path}/audio_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

#@markdown Click here if you'd like to save the diffusion model checkpoint file to your [Weights & Biases](www.wandb.ai/site) account:
save_models_to_wandb = False #@param {type:"boolean"}
save_wandb_str = '--save-wandb all' if save_models_to_wandb else ''
if save_models_to_wandb == True:
    print('Saving model checkpoints in wandb')

Google Colab not detected.


In [8]:
#@title Install dependencies
!pip install .

Processing /home/user/ai/sample-generator
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for sample-generator: filename=sample_generator-1.0.0-py3-none-any.whl size=9859 sha256=bdc7794d34d027e41e88fe216e1c6d15252cf5cf474e49abf933ee2dfc8fd9c7
  Stored in directory: /home/user/.cache/pip/wheels/3b/e8/e1/c0112ab28a1b4ead6e6884df2c52c1d04648973ae38a500521
Successfully built sample-generator
  Attempting uninstall: sample-generator
    Found existing installation: sample-generator 1.0.0
    Uninstalling sample-generator-1.0.0:
      Successfully uninstalled sample-generator-

# Train

In [4]:
#@markdown Log in to [Weights & Biases](https://wandb.ai/) for run tracking
!wandb login

wandb: Currently logged in as: felseg. Use `wandb login --relogin` to force relogin


In [33]:
#@markdown Name for the finetune project, used as the W&B project name, as well as the directory for the saved checkpoints
NAME="dd-servos-finetune" 

#@markdown Path to the directory of audio data to use for fine-tuning
TRAINING_DIR="/content/dd-servos-small/"
#@markdown Path to the checkpoint to fine-tune 
CKPT_PATH="/models/jmann-small-190k.ckpt" 

#@markdown Directory path for saving the fine-tuned outputs
OUTPUT_DIR=f"/models/{NAME}/" 

#@markdown Number of training steps between demos
DEMO_EVERY=250 
#@markdown Number of training steps between saving model checkpoints
CHECKPOINT_EVERY=500 
#@markdown Sample rate to train at
SAMPLE_RATE=48000 
#@markdown Number of audio samples per training sample
SAMPLE_SIZE=65536 
#@markdown If true, the audio samples provided will be randomly cropped to SAMPLE_SIZE samples
#@markdown
#@markdown Turn off if you want to ensure the training data always starts at the beginning of the audio files (good for things like drum one-shots)
RANDOM_CROP=True 
#@markdown Batch size to fine-tune (make it as high as it can go for your GPU)
BATCH_SIZE=2 
#@markdown Accumulate gradients over n batches, useful for training on one GPU. 
#@markdown
#@markdown Effective batch size is BATCH_SIZE * ACCUM_BATCHES.
#@markdown
#@markdown Also increases the time between demos and saved checkpoints
ACCUM_BATCHES=4

random_crop_str = f"--random-crop True" if RANDOM_CROP else ""

# Escape spaces in paths
CKPT_PATH = CKPT_PATH.replace(f" ", f"\ ")
OUTPUT_DIR = f"{OUTPUT_DIR}/{NAME}".replace(f" ", f"\ ")


ckpt_path_str = f"--ckpt-path {CKPT_PATH}" if not CKPT_PATH =="" else ""

!python3 /home/user/ai/sample-generator/train_uncond.py --ckpt-path "/home/user/ai/sample-generator/models/jmann-small-190k.ckpt" \
                                                          --name "dd-servos-smalls" \
                                                          --training-dir "/home/user/ai/sample-generator/content/dd-servos-smalls" \
                                                          --sample-size 65536 \
                                                          --accum-batches 2 \
                                                          --sample-rate 48000 \
                                                          --batch-size 1 \
                                                          --demo-every 250 \
                                                          --checkpoint-every 500 \
                                                          --num-workers 1 \
                                                          --save-wandb all \
                                                          --num-gpus 1 \
                                                          --random-crop True \
                                                          --save-path "/home/user/ai/sample-generator/models/dd-servos-smalls/"

Using device: cuda
Random crop: True
/home/user/ai/sample-generator/content/dd-servos-smalls
['/home/user/ai/sample-generator/content/dd-servos-smalls/servo_tiny_bot_transform_11_r8b_r8b--0.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_dual_operating_03_r8b--9.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_dual_operating_03_r8b--2.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_motor_tiny_bot_01_r8b--3.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_transform_sweetener_02_r8b--16.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_tiny_bot_transform_18_r8b_r8b--0.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_transform_sweetener_02_r8b_r8b--1.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_transform_sweetener_02_r8b_r8b--21.wav', '/home/user/ai/sample-generator/content/dd-servos-smalls/servo_transform_sweetener_01_r8b--5.wav', '/home/user/ai